In [ ]:
! pip install chromadb -q
! pip install sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 38

In [ ]:
import chromadb


In [ ]:
# indexing all our documents
import os
def read_files_from_folder(folder_path):
  file_data=[]

  for file_name in os.listdir(folder_path):
    if file_name.endswith(".txt"):
      with open(os.path.join(folder_path,file_name),'r') as file:
        content= file.read()
        file_data.append({"file_name":file_name,"content":content}) # name will be used as a metadata in the vector db
  return file_data

folder_path="pets"
file_data=read_files_from_folder(folder_path)

for data in file_data:
  print(f"File Name: {data['file_name']}")
  print(f"Content: {data['content']}\n")


File Name: Health Care for Pets.txt
Content: Routine health care is crucial for pets to live long, happy lives. Regular vet check-ups help catch potential issues early and keep vaccinations up to date. Dental care is also essential to prevent diseases in pets, especially in dogs and cats. Regular grooming, parasite control, and weight management are other important aspects of pet health care.

File Name: Different Types of Pet Animals.txt
Content: Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like turtles and lizards can make intriguing pets. Even fish, with their calming presence, can be wonderful pets.

File Name: The Emotional Bond Between Humans and Pets.txt
Content: Pets offer more than just co

In [ ]:
# this will add the arrays below to the collections
documents =[]
metadatas=[]
ids=[]

for index, data in enumerate(file_data):
  documents.append(data['content'])
  metadatas.append({'source':data['file_name']}) # remember metadata is a dictionary
  ids.append(str(index+1))


In [ ]:
metadatas

[{'source': 'Health Care for Pets.txt'},
 {'source': 'Different Types of Pet Animals.txt'},
 {'source': 'The Emotional Bond Between Humans and Pets.txt'},
 {'source': 'Nutrition Needs of Pet Animals.txt'},
 {'source': 'Training and Behaviour of Pets.txt'}]

In [ ]:
# we are just initializing the data
client = chromadb.PersistentClient(path="pet_db")

In [ ]:
pet_collection= client.create_collection("pet_collection")
pet_collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 70.1MiB/s]


In [ ]:
# querying
results=pet_collection.query(
    query_texts=["What are the different kinds of pets people commonly own?"],
    n_results=1
)
results

{'ids': [['2']],
 'distances': [[0.7325009209309101]],
 'metadatas': [[{'source': 'Different Types of Pet Animals.txt'}]],
 'embeddings': None,
 'documents': [['Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like turtles and lizards can make intriguing pets. Even fish, with their calming presence, can be wonderful pets.']],
 'uris': None,
 'data': None}

In [ ]:
results=pet_collection.query(
    query_texts=["What are the emotional benefits of owning a pet"], # we are not adding additional parameters
    n_results=1
)
for result in results['documents'][0]:
  print(result)
# the semantic search works fine

Pets offer more than just companionship; they provide emotional support, reduce stress, and can even help their owners lead healthier lives. The bond between pets and their owners is strong, and many people consider their pets as part of the family. This bond can be especially important in times of personal or societal stress, providing comfort and consistency.


In [ ]:
# using a different embedding model
from sentence_transformers import SentenceTransformer
model=SentenceTransformer('paraphrase-MiniLM-L3-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
documents=[]
embeddings=[]
metadatas=[]
ids=[]
for index, data in enumerate(file_data):
  documents.append(data['content'])
  embedding=model.encode(data['content']).tolist() # ensure you include tolist()
  embeddings.append(embedding)
  metadatas.append({'source': data['file_name']})
  ids.append(str(index +1 ))

In [ ]:
pet_collection_emb = client.create_collection("pet_collection_emb")
pet_collection_emb.add(
    documents=documents,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=ids
)

In [ ]:
query="What are the pets that people commonly ow?"
input_em=model.encode(query).tolist() # first create the embeddings to the query
results=pet_collection_emb.query(
    query_embeddings=[input_em],
    n_results=1
)
results

{'ids': [['2']],
 'distances': [[14.948550089417491]],
 'metadatas': [[{'source': 'Different Types of Pet Animals.txt'}]],
 'embeddings': None,
 'documents': [['Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like turtles and lizards can make intriguing pets. Even fish, with their calming presence, can be wonderful pets.']],
 'uris': None,
 'data': None}